# 1.0 Predicting heart disease using machine learning

This notebook looks into using various Python-based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has heart disease based on their medical attributes.

I am going to take the following approach:
1. Problem definition
2. Data
3. Evaluation
4. Features
5. Modelling
6. Experimentation

## 1.1 Problem Definition

In a statement,
> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

## 1.2 Data

The original data came from the Cleavland data from the UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/datasets/heart+Disease

There is also a version of it available on Kaggle. https://www.kaggle.com/ronitf/heart-disease-uci

## 1.3 Evaluation

> If the outcome can reach 95% accuracy at predicting whether or not a patient has heart disease during the proof of concept, we'll pursue the project.

## 1.4 Features

Below are the details and descriptions of the data features.

**Create data dictionary**

1. age - age in years
2. sex - (1 = male; 0 = female)
3. cp - chest pain type
    * 0: Typical angina: chest pain related decrease blood supply to the heart
    * 1: Atypical angina: chest pain not related to heart
    * 2: Non-anginal pain: typically esophageal spasms (non heart related)
    * 3: Asymptomatic: chest pain not showing signs of disease
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital) anything above 130-140 is typically cause for concern
5. chol - serum cholestoral in mg/dl
    * serum = LDL + HDL + .2 * triglycerides
    * above 200 is cause for concern
6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
    * '>126' mg/dL signals diabetes
7. restecg - resting electrocardiographic results
    * 0: Nothing to note
    * 1: ST-T Wave abnormality
        * can range from mild symptoms to severe problems
        * signals non-normal heart beat
    * 2: Possible or definite left ventricular hypertrophy
        * Enlarged heart's main pumping chamber
8. thalach - maximum heart rate achieved
9. exang - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest looks at stress of heart during excercise unhealthy heart will stress more
11. slope - the slope of the peak exercise ST segment
    * 0: Upsloping: better heart rate with excercise (uncommon)
    * 1: Flatsloping: minimal change (typical healthy heart)
    * 2: Downslopins: signs of unhealthy heart
12. ca - number of major vessels (0-3) colored by flourosopy
    * colored vessel means the doctor can see the blood passing through
    * the more blood movement the better (no clots)
13. thal - thalium stress result
    * 1,3: normal
    * 6: fixed defect: used to be defect but ok now
    * 7: reversable defect: no proper blood movement when excercising
14. target - have disease or not (1=yes, 0=no) (= the predicted attribute)

# 2.0 Preparing the Tools

In [ ]:
#!pip install sklearn
#Import all libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Inline matplotlib to view inside this notebook directly
%matplotlib inline

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

#Evaluation
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

## 2.1 Load Data

In [ ]:
df = pd.read_csv("../input/hear2csv/heart (2).csv")
df.shape

# 3.0 Data Exploration (exploratory data analysis or EDA)

The goal here is to find out more about the data and become a subject matter export on the dataset you're working with. 

1. What question(s) are you trying to solve?
2. What kind of data do we have and how do we treat different types?
3. What's missing from the data and how do you deal with it?
4. Where are the outliers and why should you care about them?
5. How can you add, change or remove features to get more out of your data?

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#how many class of one feature or target
df["target"].value_counts()

In [ ]:
#bar chart
df["target"].value_counts().plot(kind='bar', color=["salmon","lightblue"])

In this barchart, we can see that more data samples has heart disease.

In [ ]:
df.info()

In [ ]:
#check missing values of all features
df.isna().sum()

In [ ]:
df.describe()

## 3.1 Heart Disease Frequency according to Sex

In [ ]:
df.sex.value_counts()

In [ ]:
#Compare target and sex column
pd.crosstab(df.target, df.sex)

In [ ]:
#Create plot of crosstab
pd.crosstab(df.target, df.sex).plot(kind="bar",figsize=(10,6),color=["salmon","lightblue"])
plt.title("Heart Disease Frerquency for Sex")
plt.xlabel("0 = No Disease, 1=Disease")
plt.ylabel("Amount")
plt.legend(["Female","Male"]);
plt.xticks(rotation=0);

From the barchart above, the frequency of female getting heart disease is higher in this dataset compared to male. 

## 3.2 Age vs. Max Heart Rate for Heart Disease

In [ ]:
#Create new figure
plt.figure(figsize=(10,6))

#Scatter with positive examples
plt.scatter(df.age[df.target==1],
           df.thalach[df.target==1],
           c="salmon")

#Scatter with negative examples
plt.scatter(df.age[df.target==0],
           df.thalach[df.target==0],
           c="lightblue");

#Add some helpful info
plt.title("Heart Disease in function of Age and Max Heart Rate")
plt.xlabel("Age")
plt.ylabel("Max Heart Rate")
plt.legend(["Disease","No Disease"])

The chances of getting maximum heart rate is higher for heart disease patients. 

In [ ]:
#Check the distribution of the age column with a histogram
#May check for outliers of the data
df.age.plot.hist()

In this histogram, we can see that approximately half of the samples age is between 55 to 65 years old. The rest are from 40s to 70s. There are also a few samples for 30-40 and 70 above.

## 3.3 Heart Disease Frequency per Chest Pain Type
3. cp - chest pain type
    * 0: Typical angina: chest pain related decrease blood supply to the heart
    * 1: Atypical angina: chest pain not related to heart
    * 2: Non-anginal pain: typically esophageal spasms (non heart related)
    * 3: Asymptomatic: chest pain not showing signs of disease

In [ ]:
pd.crosstab(df.cp,df.target)

In [ ]:
# Make corsstab visualise
pd.crosstab(df.cp,df.target).plot(kind="bar",figsize=(10,6),color=["salmon","lightblue"])

plt.title("Heart Disease Frequency Per Chest Pain Type")
plt.xlabel("CHest Pain Type")
plt.ylabel("Amount")
plt.legend(["No Disease","Disease"])
plt.xticks(rotation=0)

Most of the heart disease patients are suffering on the third chest pain type which is non-aginal pain and some of suffering on the first the chest pain type, typical angina and atypical angina.
Although the second and third chest pain type is non related to heart, the data shows patients will suffer on those chest pain types. To make a conclusion, we might need to approach some healthcare professions to ask for their opinions.

## 3.4 Correlation
To see the relationship among the features.

In [ ]:
#Make a correlation matrix
df.corr()

In [ ]:
#Visualise correlation
corr_matrix = df.corr()
fig,ax = plt.subplots(figsize=(15,10))
ax=sns.heatmap(corr_matrix,annot=True,linewidths=0.5,fmt=".2f",cmap="YlGnBu");
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top-0.5)

1. Positive correlation, both variables increase or decrease in same direction
2. Negative correlation, one variable increase and one variable decrease vice versa

Result1: Chest pain and target have positive correlation
-> Higher chest pain level, more targets may get heart disease


# 4.0 Modelling

## Split data to training and test sets

In [ ]:
#Split data into X and y for training features and target variable
X=df.drop("target",axis=1)
y=df["target"]

In [ ]:
#Split data into train and test sets
np.random.seed(42)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

## 4.1 Train base models
### 3 different Machine Learning models:
1. Logistic Regression
2. K-Nearest Neighbors Classifiers
3. Random Forest Classifiers


In [ ]:
#Put models in a dictionary

models = {"Logistic Regression": LogisticRegression(),
         "KNN": KNeighborsClassifier(),
         "Random Forest": RandomForestClassifier()}

#Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Fits and evaluates given machine learning models.
    models : a dict of differetn Scikit-Learn machine learning models
    X_train : training data (no labels)
    X_test : testing data (no labels)
    y_train : training labels
    y_test : test labels
    """
    #set random seed
    np.random.seed(42)
    #dictionary to keep model scores
    model_scores = {}
    #loop thru models
    for name, model in models.items():
        #fit model
        model.fit(X_train, y_train)
        #evaluate model and append score
        model_scores[name]=model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)
model_scores

### 4.1.1 Base Model Comparison

In [ ]:
model_compare = pd.DataFrame(model_scores, index=["accuracy"])
model_compare.T.plot.bar()

For base model, Logistic Regression and Random Forest works way more better than KNN.

Let's look at the following:
* Hypyterparameter tuning
* Feature importance
* Confusion matrix
* Cross-validation
* Precision
* Recall
* F1 score
* Classification report
* ROC curve
* Area under the curve(AUC)

## 4.2 Hyperparameter tuning

### Ways to tune hyperparameters
1. by hand
2. RandomizedSearchCV
3. GridSearchCV

### 4.2.1 Tune by hand

In [ ]:
#Tune knn
train_scores = []
test_scores = []

#list for different values of n-neighbors
neighbors = range(1,21)

#set up knn instance
knn = KNeighborsClassifier()

#loop thru list
for i in neighbors:
    knn.set_params(n_neighbors=i)
    
    #fit the model
    knn.fit(X_train, y_train)
    
    # Update the training scores list
    train_scores.append(knn.score(X_train, y_train))
    
    # Update the test scores list
    test_scores.append(knn.score(X_test, y_test))
    

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors, train_scores, label="Train score")
plt.plot(neighbors, test_scores, label="Test score")
plt.xticks(np.arange(1, 21, 1))
plt.xlabel("Number of neighbors")
plt.ylabel("Model score")
plt.legend()

print(f"Maximum KNN score on the test data: {max(test_scores)*100:.2f}%")

After tuning the parameter for k value, KNN classifier has imporoved but the performance is still lower than Logistic Regression and Random Forest.

### 4.2.2 Hyperparameter tuning with RandomizedSearchCV

Tune: 
* LogisticRegression()
* RandomForestClassifier()

In [ ]:
#Create a hyperparameter grid for LR
log_reg_grid = {"C": np.logspace(-4,4,20),
                "solver": ["liblinear"]}

#Create a hyperparameter grid for RF
rf_grid = {"n_estimators": np.arange(10,1000,50),
          "max_depth": [None,3,5,10],
          "min_samples_split": np.arange(2,20,2),
          "min_samples_leaf": np.arange(1,20,2)}

In [ ]:
# Tune LogisticRegression

np.random.seed(42)

# Setup random hyperparameter search for LogisticRegression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model for LogisticRegression
rs_log_reg.fit(X_train, y_train)

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_lr_score = rs_log_reg.score(X_test,y_test)

In [ ]:
#Tune RF

#Set random parameter search for RF
np.random.seed(42)
rs_rf = RandomizedSearchCV(RandomForestClassifier(),
                            param_distributions=rf_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)
#Fit random hyperparameter search model for RF
rs_rf.fit(X_train,y_train)

In [ ]:
#Find best hyperparameter
rs_rf.best_params_

In [ ]:
#Evaluate the randomised search RF model
rs_rf_score =rs_rf.score(X_test, y_test)

In [ ]:
#default model scores
model_scores

By using RandomizedSearchCV, the performance of random forest model has improved. But logistic regression model's performance is still higher.

### 4.2.3 Hyperparameter Tuning with GridSearchCV
Tune: 
* LogisticRegression()

In [ ]:
#Different hyperparameters for LR model
log_reg_grid = {"C": np.logspace(-4,4,30),
               "solver": ["liblinear"]}

#Setup grid hyperparameter for LR
gs_log_reg = GridSearchCV(LogisticRegression(),
                         param_grid=log_reg_grid,
                         cv=5,
                         verbose=True)

#Fit into model
gs_log_reg.fit(X_train,y_train)

In [ ]:
#check best hyperparameters
gs_log_reg.best_params_

In [ ]:
#Evaluate grid search LR model
gs_lr_score = gs_log_reg.score(X_test,y_test)

In [ ]:
model_scores.update( [('RandomizedS LR',rs_lr_score),('RandomizedS RF',rs_rf_score),('GridS LR',gs_lr_score)] )
model_compare2 = pd.DataFrame(model_scores, index=["accuracy"])
model_compare2.T.plot.bar(legend=False)

Logistic Regression has the same scores for all of the different hyperparameter tuning own models. 

Out of three different classifiers, Logistic Regression has the best performance score during training stage.

## 4.3 Evaluationg our tuned machine learning classifier, beyond accuracy

* ROC curve and AUC score
* Confusion matrix
* Classification report
* Precision
* Recall
* F1-score

... would be great if corss-validation was used where possible

To make comparisons and evaluate our trained model, first we need to make predictions.

In [ ]:
#Make predictions with tuned model
y_preds = gs_log_reg.predict(X_test)

In [ ]:
y_preds

In [ ]:
#Plot ROC curve and calculate AUC metric
plot_roc_curve(gs_log_reg, X_test, y_test)

In [ ]:
#Confusion matrix
print(confusion_matrix(y_test, y_preds))

In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(y_test,y_preds):
    """
    Plot a nice looking confusion matrix using Seaborn's heatmap
    """
    fig,ax = plt.subplots(figsize=(3,3))
    ax=sns.heatmap(confusion_matrix(y_test,y_preds),
                  annot=True,
                  cbar=False)
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    
plot_conf_mat(y_test, y_preds)

### 4.3.1 Classification report
Cross-validated precision, recall and f1-score.

In [ ]:
#Using only one split
print(classification_report(y_test,y_preds))

#### Calculate evaluation metrics using cross-validation
using cross_val_score()

In [ ]:
#Check best hyperparameters
gs_log_reg.best_params_

In [ ]:
#Create a new classifier with best parameters
clf = LogisticRegression(C=0.20433597178569418,
                        solver="liblinear")

In [ ]:
#Cross validated accuracy
cv_acc= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="accuracy")
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
#Cross-validated precision
cv_precision= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="precision")
cv_precision = np.mean(cv_precision)
cv_precision

In [ ]:
#Cross-validated recall
cv_recall= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="recall")
cv_recall = np.mean(cv_recall)
cv_recall

In [ ]:
#Cross-validated f1
cv_f1= cross_val_score(clf,
                       X,
                       y,
                       cv=5,
                       scoring="f1")
cv_f1 = np.mean(cv_f1)
cv_f1

In [ ]:
#Visualise cross-validated metrics
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                          "Precision": cv_precision,
                          "Recall": cv_recall,
                          "F1": cv_f1},
                          index=[0])

cv_metrics.T.plot.bar(title="Cross-validated classification metrics",
                      legend=False);

# 5.0 Experimentation

For furthur experimentation and training:

* Collect more data?
* Try a better model? Like CatBoost or XGBoost?
* Improve the current models? 

## 5.1 Feature Importance

Feature importance is another as asking, "which features contributed most to the outcome of the model and how did they contribute?"

Finding feature importance is different for each machine learning model.

We may refer to feature importance for future collecting data.


In [ ]:
#Fit an instance of LR
clf = LogisticRegression(C=0.20433597178569418,
                        solver="liblinear")

clf.fit(X_train,y_train);

In [ ]:
#Check coef_
clf.coef_

In [ ]:
#Match coef to columns
feature_dict = dict(zip(df.columns, list(clf.coef_[0])))
feature_dict

In [ ]:
#Visualise feature importance
feature_df = pd.DataFrame(feature_dict,index=[0])
feature_df.T.plot.bar(title="Feature Importance",legend=False)

Base on the visualisation, 
* chest pain type(cp)
* resting electrocardiographic results(restecg) 
* slope of the peak exercise ST segment(slope) 

have strong feature importance.

On the other hand, sex has the least feature importance.